In [59]:
import pandas as pd 
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET


In [60]:
url = r"C:\Users\honan\OneDrive\Desktop\test forecast load 3d.csv"



In [61]:
source_file = pd.read_csv(url)

source_file.head()


,tablescraper-selected-row,tablescraper-selected-row href,table-results__name,tablescraper-selected-row 2,tablescraper-selected-row 3
0,PUB_DailyLoadFcst_202304031435.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:35:35
1,PUB_DailyLoadFcst_202304031420.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:20:46
2,PUB_DailyLoadFcst_202304031415.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:15:15
3,PUB_DailyLoadFcst_202304031405.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:05:16
4,PUB_DailyLoadFcst_202304031400.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:00:56


In [62]:
# rename columns to shorturl, fullurl, name, trade_date, and publish_date
source_file.columns = ['shorturl', 'fullurl', 'name', 'trade_date', 'publish_date']

source_file.head()
 

,shorturl,fullurl,name,trade_date,publish_date
0,PUB_DailyLoadFcst_202304031435.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:35:35
1,PUB_DailyLoadFcst_202304031420.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:20:46
2,PUB_DailyLoadFcst_202304031415.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:15:15
3,PUB_DailyLoadFcst_202304031405.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:05:16
4,PUB_DailyLoadFcst_202304031400.xml,https://reports.sem-o.com/documents/PUB_DailyL...,Daily Load Forecast Summary,2023-04-03,2023-04-03T14:00:56


In [63]:
import pandas as pd
import requests
from xml.etree import ElementTree as ET
from tqdm import tqdm


def parse_xml_to_df(xml_content):
    tree = ET.fromstring(xml_content)
    data = []
    publish_time = tree.get('PublishTime')
    for elem in tree.findall('PUB_DailyLoadFcst'):
        row = {'PublishTime': publish_time}
        for child in elem:
            row[child.tag] = child.text
        data.append(row)
    return pd.DataFrame(data)

dfs = []

for url in tqdm(source_file['fullurl'], desc="Reading files"):
    response = requests.get(url)
    if response.status_code == 200:
        xml_content = response.content
        df = parse_xml_to_df(xml_content)
        dfs.append(df)

final_df = pd.concat(dfs, ignore_index=True)





Reading files: 100%|██████████| 1080/1080 [01:52<00:00,  9.59it/s]


In [91]:
import plotly.graph_objs as go

final_df['PublishTime'] = pd.to_datetime(final_df['PublishTime'])

# Sort the DataFrame 
final_df.sort_values('AggregatedForecast', inplace=True)

# Convert PublishTime to ordinal value
final_df['PublishTimeOrdinal'] = final_df['PublishTime'].apply(lambda x: x.toordinal())

# Create 3d plot
fig = go.Figure(data=[go.Scatter3d(
    x=final_df['StartTime'],
    y=final_df['AggregatedForecast'],
    z=final_df['PublishTime'],
    mode='markers',
    marker=dict(
        size=3,
        color=final_df['PublishTimeOrdinal'],
        colorscale='Viridis',
        opacity=0.8
    )
)])

# configure axis
fig.update_layout(scene=dict(
                    xaxis_title='Start Time',
                    yaxis_title='Aggregated Forecast',
                    zaxis_title='Publish Time',
                    ),
                    width=900,
                    height=900,
                    title='3D Scatter plot of Aggregated Load Forecast Data')

fig.show()


In [85]:
# convert the dataframe to a csv file
final_df.to_csv(r'C:\Users\honan\OneDrive\Desktop\test forecast load 3.csv', index = False)

# open the csv file in excel
import os
os.startfile(r'C:\Users\honan\OneDrive\Desktop\test forecast load 3.csv')